In [1]:
import numpy as np
import pandas as pd 

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
# read data from csv
traind= pd.read_csv("./data/train.csv")
testd= pd.read_csv("./data/test.csv")

In [4]:
traind.head()

,id,Gender,Age,Driving_License,Region_Code,Previously_Insured,Vehicle_Age,Vehicle_Damage,Annual_Premium,Policy_Sales_Channel,Vintage,Response
0,1,Male,44,1,28.0,0,> 2 Years,Yes,40454.0,26.0,217,1
1,2,Male,76,1,3.0,0,1-2 Year,No,33536.0,26.0,183,0
2,3,Male,47,1,28.0,0,> 2 Years,Yes,38294.0,26.0,27,1
3,4,Male,21,1,11.0,1,< 1 Year,No,28619.0,152.0,203,0
4,5,Female,29,1,41.0,1,< 1 Year,No,27496.0,152.0,39,0


In [5]:
traind.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 381109 entries, 0 to 381108
Data columns (total 12 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   id                    381109 non-null  int64  
 1   Gender                381109 non-null  object 
 2   Age                   381109 non-null  int64  
 3   Driving_License       381109 non-null  int64  
 4   Region_Code           381109 non-null  float64
 5   Previously_Insured    381109 non-null  int64  
 6   Vehicle_Age           381109 non-null  object 
 7   Vehicle_Damage        381109 non-null  object 
 8   Annual_Premium        381109 non-null  float64
 9   Policy_Sales_Channel  381109 non-null  float64
 10  Vintage               381109 non-null  int64  
 11  Response              381109 non-null  int64  
dtypes: float64(3), int64(6), object(3)
memory usage: 34.9+ MB


In [7]:
unseen=traind.sample(frac=.05,random_state=42)
train=traind.drop(unseen.index)
unseen.to_csv('./data/unseen.csv',index=False)
train.to_csv('./data/train.csv',index=False)
unseen.shape,train.shape

((19055, 12), (362054, 12))

In [8]:
from pycaret.classification import ClassificationExperiment
test=ClassificationExperiment()

In [10]:
test.setup(train,target='Vehicle_Damage',session_id=66)

,Description,Value
0,Session id,66
1,Target,Vehicle_Damage
2,Target type,Binary
3,Target mapping,"No: 0, Yes: 1"
4,Original data shape,"(362054, 12)"
5,Transformed data shape,"(362054, 14)"
6,Transformed train set shape,"(253437, 14)"
7,Transformed test set shape,"(108617, 14)"
8,Numeric features,9
9,Categorical features,2


In [11]:
test.compare_models()

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
ada,Ada Boost Classifier,0.9132,0.0000,0.9132,0.9152,0.9130,0.8262,0.8283,7.1110
gbc,Gradient Boosting Classifier,0.9132,0.0000,0.9132,0.9153,0.9131,0.8263,0.8284,24.7520
lightgbm,Light Gradient Boosting Machine,0.9132,0.0000,0.9132,0.9153,0.9130,0.8262,0.8284,3.2740
catboost,CatBoost Classifier,0.9130,0.0000,0.9130,0.9151,0.9128,0.8258,0.8280,50.5610
rf,Random Forest Classifier,0.9114,0.0000,0.9114,0.9131,0.9112,0.8226,0.8244,26.4840
lr,Logistic Regression,0.9111,0.0000,0.9111,0.9134,0.9110,0.8221,0.8245,15.2970
ridge,Ridge Classifier,0.9111,0.0000,0.9111,0.9134,0.9110,0.8221,0.8245,1.1780
lda,Linear Discriminant Analysis,0.9111,0.0000,0.9111,0.9134,0.9110,0.8221,0.8245,1.2840
et,Extra Trees Classifier,0.9104,0.0000,0.9104,0.9120,0.9103,0.8207,0.8223,16.5900
dt,Decision Tree Classifier,0.8463,0.0000,0.8463,0.8463,0.8463,0.6925,0.6926,2.5190


AdaBoostClassifier(algorithm='SAMME.R', estimator=None, learning_rate=1.0,
                   n_estimators=50, random_state=66)

In [14]:
ada=test.create_model('ada')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.9124,0.0000,0.9124,0.9142,0.9122,0.8246,0.8265
1,0.9164,0.0000,0.9164,0.9182,0.9163,0.8327,0.8345
2,0.9131,0.0000,0.9131,0.9152,0.9129,0.8260,0.8282
3,0.9132,0.0000,0.9132,0.9151,0.9130,0.8262,0.8282
4,0.9108,0.0000,0.9108,0.9129,0.9107,0.8215,0.8237
5,0.9129,0.0000,0.9129,0.9149,0.9127,0.8256,0.8277
6,0.9124,0.0000,0.9124,0.9145,0.9123,0.8248,0.8269
7,0.9115,0.0000,0.9115,0.9134,0.9114,0.8229,0.8248
8,0.9137,0.0000,0.9137,0.9158,0.9136,0.8274,0.8295


In [16]:
tuned_ada=test.tune_model(ada)
tuned_ada

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.9098,0.0000,0.9098,0.9120,0.9097,0.8195,0.8218
1,0.9138,0.0000,0.9138,0.9159,0.9137,0.8275,0.8297
2,0.9113,0.0000,0.9113,0.9136,0.9111,0.8225,0.8248
3,0.9113,0.0000,0.9113,0.9135,0.9111,0.8224,0.8246
4,0.9088,0.0000,0.9088,0.9112,0.9086,0.8175,0.8199
5,0.9108,0.0000,0.9108,0.9131,0.9107,0.8215,0.8238
6,0.9104,0.0000,0.9104,0.9127,0.9102,0.8206,0.8230
7,0.9096,0.0000,0.9096,0.9117,0.9094,0.8190,0.8212
8,0.9120,0.0000,0.9120,0.9143,0.9119,0.8239,0.8262


Fitting 10 folds for each of 10 candidates, totalling 100 fits
Original model was better than the tuned model, hence it will be returned. NOTE: The display metrics are for the tuned model (not the original one).


AdaBoostClassifier(algorithm='SAMME.R', estimator=None, learning_rate=1.0,
                   n_estimators=50, random_state=66)

In [17]:
final_ada=test.finalize_model(tuned_ada)
final_ada

Pipeline(memory=Memory(location=None),
         steps=[('label_encoding',
                 TransformerWrapperWithInverse(exclude=None, include=None,
                                               transformer=LabelEncoder())),
                ('numerical_imputer',
                 TransformerWrapper(exclude=None,
                                    include=['id', 'Age', 'Driving_License',
                                             'Region_Code',
                                             'Previously_Insured',
                                             'Annual_Premium',
                                             'Policy_Sales_Channel', 'Vintage',
                                             'Response'],
                                    transformer=Si...
                 TransformerWrapper(exclude=None, include=['Vehicle_Age'],
                                    transformer=OneHotEncoder(cols=['Vehicle_Age'],
                                                              drop_invariant=False,
                                                              handle_missing='return_nan',
                                                              handle_unknown='value',
                                                              return_df=True,
                                                              use_cat_names=True,
                                                              verbose=0))),
                ('actual_estimator',
                 AdaBoostClassifier(algorithm='SAMME.R', estimator=None,
                                    learning_rate=1.0, n_estimators=50,
                                    random_state=66))],
         verbose=False)

In [19]:
unseen_X=unseen.copy().drop('Vehicle_Damage',axis=1)
y=unseen['Vehicle_Damage']

prediction=test.predict_model(final_ada,data=unseen_X)

In [20]:
from sklearn.metrics import classification_report

# Use classification_report to compare predicted values with actual values
y_pred=prediction['prediction_label']
print(classification_report(y,y_pred))

              precision    recall  f1-score   support

          No       0.94      0.88      0.91      9380
         Yes       0.89      0.95      0.92      9675

    accuracy                           0.92     19055
   macro avg       0.92      0.91      0.91     19055
weighted avg       0.92      0.92      0.92     19055



## Result ##
* For the '0' Response, the precision is 0.87, indicating that 87% of instances predicted as '0' are indeed '0'. For the '1' Response, is 0.00
* For the '0' Response, the recall is 1.00, indicating that the model successfully predicts 100% of actual '0' instances. For the '1' Response, the recall is 0.00, meaning that the model correctly predicts 0% of actual '1' instances.
* It was determined that 1 refers to a non-responding customer. So there is no data generated per se, so it is '0'.
* Overall, the model performs well in predicting the '1' but performs relatively poorly in predicting the '1'.